In [ ]:
###step 0: imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as pdr
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import plotting
###step 2: load in variables
#lists of stocks
bottomstocksList = ['LEG' , 'RL' , 'HBI' , 'IPGP', 'FOX', 'GPS' , 'UAA', 'DISCA' , 'UA' , 'NWS']
#1)Leggett & Platt 2) Ralph Lauren Corporation 3) Hanesbrands Inc. 4) IPG Photonics Corporation 5)Fox Corporation
#6)The Gap, Inc. 7)Under Armour, Inc. 8)Discovery, Inc. 9) Under Armour, Inc. 10) News Corporation

topstocksList = ['AAPL', 'MSFT' , 'AMZN' , 'TSLA' , 'GOOGL', 'FB', 'NVDA', 'BRK-B', 'JPM' , 'JNJ']
#1)Apple Inc.  2) 3) 4) 5)
#6) 7) 8) 9) 10)

cryptoList = ['ADA', 'SOL1' , 'DOT1' , 'MIOTA', 'EOS' , 'XLM', 'AVAX', 'ATOM1', 'ALGO', 'SHIB']
#
cryptos = [crypto + '-USD' for crypto in cryptoList]

commodities = ['CL=F', 'NG=F', 'GC=F', 'SI=F','LE=F', 'ZC=F', 'KC=F', 'SB=F', 'HG=F', 'ZW=F'] 
#1)crude oil 2)natural gas 3)gold 4)silver 5)live cattle 6)corn 7)coffee  8)sugar 9)copper 10) wheat

#date range of past year
endDate = dt.datetime(2021, 11, 18)
startDate = endDate - dt.timedelta(days=365)

#groups of interest
group_top = topstocksList
group_bttm = bottomstocksList
group_top_commd = topstocksList+commodities
group_bttm_commd = bottomstocksList+commodities
group_top_crypt = topstocksList+cryptos
group_bttm_crypt = bottomstocksList+cryptos

groups = [group_bttm, group_bttm_commd, group_bttm_crypt, group_top, group_top_commd, group_top_crypt]
group_name = ["Bottom Stocks", "Bottom Stocks & Commodities", "Bottom Stocks & Crypts", "Top Stocks", "Top Stocks & Commodities", "Top Stocks & Cryptos"]

In [ ]:
###step 2: load in functions

#function to pull stock data from yahoo finance
#given a list of stocks returns their mean returns, covariance and correlation
def get_data(stocks, start, end):
    stockData = pdr.DataReader(stocks, 'yahoo', start, end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    corMatrix = returns.corr()
    return meanReturns, covMatrix, corMatrix

#load a set of data
def load_data(group_idx = 0):
  meanReturns, covMatrix, corMatrix = get_data(groups[group_idx], startDate, endDate)
  return meanReturns, covMatrix, corMatrix

#calculates optimal weights and plots their distribution
def get_optimal_weights(group_idx, meanReturns, covMatrix):
  ef = EfficientFrontier(meanReturns*180, covMatrix*180)
  weights = ef.max_sharpe()
  cleaned_weights = ef.clean_weights()
  cleaned_weights =  list(cleaned_weights.items())
  cleaned_weights = [y for (x,y) in cleaned_weights]

  #plot weight distribution
  fig = plt.figure()
  ax = fig.add_axes([0,0,1,1])
  ax.bar(groups[group_idx], cleaned_weights, color=['blue']*10+['green']*10)
  plt.xticks(rotation = 45) # Rotates X-Axis Ticks by 45-degrees

  title_string = "Optimal Weight Distribution (" + group_name[group_idx] + ")"
  plt.title(title_string)
  file_name = title_string + ".png"
  #plt.savefig(file_name)
  plt.savefig(file_name,bbox_inches='tight', dpi=150)
  return cleaned_weights

#graphs efficient frontier plot
def graph_efficiency_frontier(group_idx, meanReturns, covMatrix):
  ef = EfficientFrontier(meanReturns*180, covMatrix*180, solver="SCS")
  fig, ax = plt.subplots()
  plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)

  # Find the tangency portfolio
  ef = EfficientFrontier(meanReturns*180, covMatrix*180)
  ef.max_sharpe()
  ret_tangent, std_tangent, _ = ef.portfolio_performance()
  ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

  #run a monte carlo simulation of portfolio weights
  n_samples = 1000
  w = np.random.dirichlet(np.ones(len(meanReturns)), n_samples)
  rets = w.dot(meanReturns*180)
  stds = np.sqrt(np.diag(w @ covMatrix*180 @ w.T))
  sharpes = rets / stds
  ax.scatter(stds, rets, marker=".", c=sharpes, cmap="RdYlGn")

  #mark lowest volatility with blue diamond
  min_std_i = np.argmin(stds)
  plt.scatter(stds[min_std_i],rets[min_std_i],marker="D",color='b',s=80, label='Lowest Volatility')
  #mark max sharpe ratio with red diamond
  max_sha_i = np.argmax(sharpes)
  plt.scatter(stds[max_sha_i],rets[max_sha_i],marker="D",color='r',s=80, label='Best Sharpe Ratio')
  #mark equal distribution with green diamond
  equal_weights = np.ones(len(meanReturns)) / len(meanReturns)
  eq_return = (meanReturns*180)@equal_weights
  eq_std_dev = np.sqrt(equal_weights@(covMatrix*180)@equal_weights)
  plt.scatter(eq_std_dev, eq_return, marker="D", color='g', s=80, label='Equal Distribution')

  #add legend
  plt.legend(loc=4)

  #title and save graph
  title_string = "Efficient Frontier with random Portfolios (" + group_name[group_idx] + ")"
  plt.title(title_string)
  file_name = title_string + ".png"
  plt.savefig(file_name, bbox_inches = 'tight')
  plt.cla()
  plt.clf()
def get_GHHI(weights, corMatrix):
  #GHHI diversity (uses weights and correlation matrix)
  num_items = len(weights)
  HHI = 0
  GHHI = 0
  for i in range(num_items):
      HHI += (weights[i] ** 2)
      for j in range(num_items):
          if (i == j):
            continue
          GHHI += weights[i] * weights[j] * 2
          corMatrix.iat[i,j]
  return (GHHI+HHI)

def get_projected_Sharpe(weights, meanReturns, covMatrix):
  #projected sharpe ratio
  portfolio_return =  np.sum(meanReturns*weights)*180
  portfolio_std_dev = np.sqrt(weights@(covMatrix)@weights)*np.sqrt(180)
  s = (portfolio_return - 0) / portfolio_std_dev
  return s

def run_Monte_Carlo(group_idx, weights, meanReturns, covMatrix):
  # Monte Carlo Simulation of Price
  mc_sims = 4000 # number of simulations
  T = 180 #timeframe in days

  meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
  meanM = meanM.T

  portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)

  initialPortfolio = 10000 #value (dollars)

  for m in range(0, mc_sims):
      Z = np.random.normal(size=(T, len(weights)))#uncorrelated RV's
      L = np.linalg.cholesky(covMatrix) #Cholesky decomposition to Lower Triangular Matrix
      dailyReturns = meanM + np.inner(L, Z) #Correlated daily returns for individual stocks
      portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)*initialPortfolio

  plt.plot(portfolio_sims)
  plt.ylabel('Portfolio Value ($)')
  plt.xlabel('Days')
  title_string = "MC Portfolio Simulation (" + group_name[group_idx] + ")"
  plt.title(title_string)
  file_name = title_string + ".png"
  plt.savefig(file_name,  bbox_inches = 'tight')

  #calculate values to return
  #returns (uses total on last day and initialPortfolio value)
  total_returns = np.zeros(mc_sims)
  for i in range(mc_sims):
      total_returns[i] = (portfolio_sims[-1,i] - initialPortfolio) / initialPortfolio

  #risk (uses returns, and standard deviation)\
  risk_free = 0
  sharpe_ratio = (total_returns.mean() - risk_free) / total_returns.std()

  return total_returns.mean(), total_returns.std(), sharpe_ratio

In [ ]:
#run full simulation for all groups
for i in range(len(groups)):
  meanReturns, covMatrix, corMatrix = load_data(i)

  optimal_weights = get_optimal_weights(i, meanReturns, covMatrix)

  graph_efficiency_frontier(i, meanReturns, covMatrix)

  GHHI = get_GHHI(optimal_weights, corMatrix)
  
  projected_sharpe_ratio = get_projected_Sharpe(optimal_weights, meanReturns, covMatrix)

  mean_returns, volatility, sharpe_ratio = run_Monte_Carlo(i, optimal_weights, meanReturns, covMatrix)

  with open('results.txt', 'a') as f:
      f.write('\n\nData for ')
      f.write(group_name[i])
      f.write('\nOptimal Weights: [')
      for w in optimal_weights:
        f.write("%s," % w)
      f.write(f"]\nGHHI: {GHHI}\n")
      f.write(f"Projected Sharpe Ratio: {projected_sharpe_ratio}\n")
      f.write(f"Monte Carlo Results \nReturns: {mean_returns} Volatility: {volatility} Sharpe ratio: {sharpe_ratio}\n")
